In [29]:
# Instalando dependencias 
%pip install --quiet wget
%pip install --quiet duckdb
%pip install --quiet jupysql
%pip install --quiet duckdb-engine
%pip install --quiet pandas
%pip install --quiet matplotlib
%pip install --quiet psycopg2-binary
%pip install --quiet dash
%pip install --quiet plotly

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [23 lines of output]
      running egg_info
      creating /tmp/pip-pip-egg-info-msrvuc1t/psycopg2.egg-info
      writing /tmp/pip-pip-egg-info-msrvuc1t/psycopg2.egg-info/PKG-INFO
      writing dependency_links to /tmp/pip-pip-egg-info-msrvuc1t/psycopg2.egg-info/dependency_links.txt
      writing top-level names to /tmp/pip-pip-egg-info-msrvuc1t/psycopg2.egg-info/top_level.txt
      writing manifest file '/tmp/pip-pip-egg-info-msrvuc1t/psycopg2.egg-info/SOURCES.txt'
      
      E

In [30]:
import wget
import duckdb
import pandas as pd

In [78]:
#DC Mayor's office of Planning and Economic Development - Dataset

df = pd.read_csv('dataset.csv', delimiter=";", encoding='ISO-8859-1')

# Substituindo espaços por underlines nos nomes das colunas
df.columns = df.columns.str.replace(' ', '_').str.lower()

# Removendo o '$' e ';' e convertendo para float
df['official_amount'] = df['official_amount'].str.replace('[$;]', '', regex=True).astype(float)

In [79]:
df.tail()

,prospect_name,great_streets_corridor,official_amount,fiscal_year,business_address,city,state,zip_code,naic_codes,business_type,ward,latitude,longitude,total_number_of_projected/retained_jobs,total_projected_number_of_jobs,total_number_of_employees_retained
215,El Tamarindo; Inc. dba El Tamarindo,U Street/14th Street NW,37500.0,2016.0,1785 Florida Avenue NW,Washington,DC,20009.0,72,Restaurant,Ward 1,38.91719,-77.04129,30.0,11.0,19.0
216,Purple Patch; LLC dba Purple Patch Restaurant,U Street/14th Street NW,40000.0,2016.0,3155 Mount Pleasant Street NW,Washington,DC,20010.0,72,Restaurant,Ward 1,38.93057,-77.03768,60.0,40.0,20.0
217,Le Bustiere Boutique; LLC,U Street/14th Street NW,25000.0,2016.0,1744 Columbia Rd NW; Unit # 2,Washington,DC,20002.0,44-45,Retail,Ward 1,38.92383,-77.04047,8.0,5.0,3.0
218,Rachel Joyce Studios; LLC dba Rachel Joyce Org...,U Street/14th Street NW,18477.0,2016.0,1754 Columbia Road NW #2,Washington,DC,20009.0,81,Salon,Ward 1,38.92365,-77.04068,16.0,12.0,4.0
219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN


In [80]:
# Carregando e configurando o Jupyter Notebook para trabalhar com o DuckDB 
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%config SqlMagic.named_parameters=True

# Connect to a local DuckDB instance
%sql duckdb:///

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


/home/airtonlirajr/.local/lib/python3.10/site-packages/sql/traits.py:20: FutureWarning: named_parameters: boolean values are now deprecated. Value True will be treated as "enabled". 
Please use a valid option: "warn", "enabled", or "disabled". 
For more information, see the docs: https://jupysql.ploomber.io/en/latest/api/configuration.html#named-parameters
  warnings.warn(


In [81]:
# Conectando ao DuckDB
con = duckdb.connect(database=':memory:', read_only=False)

# Registrando o DataFrame do Pandas como uma tabela no DuckDB
con.register('my_table', df)

# Agora você pode executar consultas SQL na tabela registrada
result = con.execute("SELECT * FROM my_table").fetchdf()

print(result)

                                         prospect_name  \
0          Ask Investment Corp dba Dollar Plus Savings   
1               Compass Coffee; LLC dba Compass Coffee   
2                                          Ivy & Coney   
3                               Gwynn Inc. dba Halcyon   
4              Henderson Enterprise dba Wanda's on 7th   
..                                                 ...   
215                El Tamarindo; Inc. dba El Tamarindo   
216      Purple Patch; LLC dba Purple Patch Restaurant   
217                          Le Bustiere Boutique; LLC   
218  Rachel Joyce Studios; LLC dba Rachel Joyce Org...   
219                                               None   

          great_streets_corridor  official_amount  fiscal_year  \
0    7th St NW/Georgia Avenue NW         85000.00       2013.0   
1    7th St NW/Georgia Avenue NW         85000.00       2014.0   
2    7th St NW/Georgia Avenue NW         85000.00       2014.0   
3    7th St NW/Georgia Avenue NW       

In [82]:
import random
import psycopg2

# Conexão com o banco de dados PostgreSQL
conn = psycopg2.connect(
    dbname='postgres', 
    user='admin', 
    password='admin', 
    host='localhost', 
    port='5432'
)

In [83]:
from psycopg2 import sql

cur = conn.cursor()

# Criando a tabela dataset
cur.execute("""
CREATE TABLE IF NOT EXISTS dataset (
    id SERIAL PRIMARY KEY,
    city VARCHAR(255),
    official_amount FLOAT
)
""")

# Lista de cidades americanas para inserir na tabela
cities = ["New York", "Los Angeles", "Chicago", "Houston", "Phoenix",
          "Philadelphia", "San Antonio", "San Diego", "Dallas", "San Jose",
          "Austin", "Jacksonville", "Fort Worth", "Columbus", "Charlotte",
          "San Francisco", "Indianapolis", "Seattle", "Denver", "Washington"]

# Inserindo 50 registros na tabela
for _ in range(50):
    city = random.choice(cities)
    official_amount = round(random.uniform(1000, 5000), 2)
    cur.execute(
        sql.SQL("INSERT INTO dataset (city, official_amount) VALUES (%s, %s)"),
        [city, official_amount]
    )

# Commitando a transação
conn.commit()

# Fechando a conexão
cur.close()
conn.close()

In [84]:
%%sql

select * from postgres.public.dataset
limit 5;

,id,city,official_amount
0,1,Fort Worth,3250.56
1,2,San Francisco,4603.75
2,3,Austin,2559.80
3,4,Seattle,3315.64
4,5,San Francisco,2264.39


In [93]:
%%sql


# Caso ja exista conexão com postgresql
DETACH DATABASE pg;

# Conectando com postgresql
ATTACH 'dbname=postgres user=admin host=localhost password=admin connect_timeout=10' AS pg (TYPE postgres, READ_ONLY);

# Executando a consulta de junção no DuckDB
SELECT 
    A.city, 
    SUM(A.official_amount) AS total_official_amount
FROM postgres.public.dataset AS A
INNER JOIN my_table AS B USING (city)
GROUP BY A.city


,city,total_official_amount
0,Washington,1254764.88
